In [ ]:
!pip install gym_super_mario_bros==7.3.0 nes_py

In [ ]:
#Importing the game
import gym_super_mario_bros
#Importing the Joypad wrapper
from nes_py.wrappers import JoypadSpace
#Importing the SIMPLIFIED controls
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT

In [ ]:
# Setting up the game
env = gym_super_mario_bros.make('SuperMarioBros-v0',apply_api_compatibility=True,render_mode="human" )
#Joy pad space allows the number of options AI has down to less than 10 which makes it easier for the AI 
env = JoypadSpace(env, SIMPLE_MOVEMENT)

In [ ]:
#Create a flag - restart or not
done = True
#step represents each from. Loop through each frame in the game
for step in range(100000):
    #Start the game to begin with
    if done:
        #resets game(Starts the game)
        env.reset()
    #Do random actions
    state, reward, done, info = env.step(env.action_space.sample())
    #Show the game on the screen
    env.render()
#Close the game
env.close()

In [ ]:
#Install pytorch
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

In [ ]:
#INstall stable baselines for RL stuff
!pip install stable-baselines3[extra]

In [ ]:
#Import Frame Stacker Wrapper and GrayScaling Wrapper
from gym.wrappers import GrayScaleObservation
#Import Vectorization Wrappers
from stable_baselines3.common.vec_env import VecFrameStack, DummyVecEnv
#Import Matplotlib to show the impact of frame stacking
from matplotlib import pyplot as plt

In [ ]:
# 1. Create the base enviornment
JoypadSpace.reset = lambda self, **kwargs: self.env.reset(**kwargs)
env = gym_super_mario_bros.make('SuperMarioBros-v0',apply_api_compatibility=True,render_mode="human" )
# 2. Simplify the controls
env = JoypadSpace(env, SIMPLE_MOVEMENT)
# 3. Grayschale
env = GrayScaleObservation(env, keep_dim=True)
# 4. Wrap inside the Dummy environment
env = DummyVecEnv([lambda: env])
# 5. Stack the frames
env = VecFrameStack(env, 4, channels_order='last')

In [ ]:
#train the RL model
#Import os for file path management
import os
#Import PPO for algos
from stable_baselines3 import PPO
#Import Base Callback for saving models
from stable_baselines3.common.callbacks import BaseCallback

In [ ]:
class TrainAndLoggingCallback(BaseCallback):

    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'best_model_{}'.format(self.n_calls))
            self.model.save(model_path)

        return True
                

In [ ]:
CHECKPOINT_DIR = './train/'
LOG_DIR = './logs/'

In [ ]:
#Setup model saving callback
callback = TrainAndLoggingCallback(check_freq=10000, save_path = CHECKPOINT_DIR)

In [ ]:
#Starting the AI model
model = PPO('CnnPolicy', env, verbose =1, tensorboard_log=LOG_DIR, learning_rate=0.000001, n_steps=512)

In [ ]:
#Train model
model.learn(total_timesteps = 1000000, callback=callback)

In [ ]:
#Test model
model = PPO.load('./train/best_model_1000000')

In [ ]:
state = env.reset()
while True:
    action, _ = model.predict(state)
    state, rewards, done, info = env.step(action)
    env.render()
env.close()